In [ ]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

from models.worker import Worker
from models.outside_expert import OutsideExpert
from models.user import User
            
file_paths = ['./data/helpful-base/test.jsonl', './data/harmless-base/test.jsonl']

model_id = 'princeton-nlp/Sheared-LLaMA-2.7B-ShareGPT'
# model_id = 'mistralai/Mistral-7B-Instruct-v0.2'

In [ ]:
worker = Worker(model_id)
worker.model

In [ ]:
worker.check_device_map(no_split_module_classes=["MistralDecoderLayer"])

In [ ]:
worker.load_model()

In [ ]:
# Could make these from the training set with the preferred responses (slice them off so you don't fine tune on them?)

examples = [
    {
        "query":"What is the capital of Japan?",
        "answer":"Tokyo is the capital city of Japan."
    },
    {
        "query":"What is the main ingredient in an omlette?",
        "answer":"The main ingredient in an omelette is egg."
    }
]

worker.set_prompt_examples(examples)

In [ ]:
user_data_path = './data/harmless-base/test.jsonl'

user = User(data_path=user_data_path)

In [ ]:
num_queries = 2

queries = []

for n in range(num_queries):
    queries.append(user.get_query())

responses = worker.generate_text(queries)
for response in responses:
    print(response)

In [ ]:
expert_model_id = 'gpt-4'

openai_key_path = 'keys/openai_key.txt'

with open(openai_key_path, 'r') as file:
    openai_key = file.read().strip()

expert_example = {'example_query':'What is the capital of Japan?',
                  'example_answer':'Sausages',
                  'example_rating':0}

helpfulness_expert = OutsideExpert(
    model_id=expert_model_id,
    openai_key=openai_key,
    target='helpfulness',
    example=expert_example)

helpfulness_expert.rate_responses(queries, responses)